#                     Facial expression recognition for offline videos

Check your tensorflow version. This notebook will require TF-2.6.0

In [10]:
import tensorflow as tf
print("Tensorflow version:", tf.__version__)

Tensorflow version: 2.6.0


If you don't have TF-2.6.0 run the next cell to uninstall the old tensorflow. Else skip.

In [ ]:
!pip uninstall tensorflow -y
!pip uninstall tensorflow-estimator -y
!pip uninstall gast -y
!pip uninstall tensorboard -y
!pip uninstall grpcio -y
!pip uninstall jinja2 -y
!pip uninstall h5py -y
!pip install tensorflow==2.5
!pip install keras==2.4

In [8]:
import os
os.system('pip install seaborn')
os.system('pip install matplotlib')
os.system('pip install livelossplot')
os.system('pip install ipython')
os.system('pip install tensorflow==2.5')
os.system('pip install tensorflow==2.4')
os.system('pip install numpy')
os.system('sudo apt-get update')
os.system('sudo apt-get install ffmpeg libsm6 libxext6  -y')
os.system('pip install opencv-python')
os.system('pip install h5py==2.10.0')

0

In [3]:
import keras
print("keras version:", keras.__version__)

keras version: 2.6.0


In [2]:
import tensorflow as tf
print("Tensorflow version:", tf.__version__)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras.callbacks import ModelCheckpoint
import cv2
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
import numpy as np
from keras.preprocessing.image import img_to_array
from keras.models import load_model


Tensorflow version: 2.6.0


### Loading the training and testing data

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

img_size = 48
plt.figure(0, figsize=(12,20))
ctr = 0

for expression in os.listdir('facial_expression_recognition/train'):
    for i in range(1,6):
        ctr += 1
        plt.subplot(7,5,ctr)
        img = load_img("facial_expression_recognition/train" + expression + "/" 
                       +os.listdir("facial_expression_recognition/train" + expression)[i], target_size=(img_size, img_size))
        plt.imshow(img, cmap="gray")

plt.tight_layout()
plt

Data Augmentation for training and validation images

In [ ]:
img_size = 48
batch_size = 64

# Data generator to augment data for training
datagen_train = ImageDataGenerator(horizontal_flip=True)
train_generator = datagen_train.flow_from_directory("facial_expression_recognition/train/",
                                                    target_size=(img_size,img_size),
                                                    color_mode='grayscale',
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)

# Data generator to augment data for validation
datagen_validation = ImageDataGenerator(horizontal_flip=True)
validation_generator = datagen_train.flow_from_directory("facial_expression_recognition/test/",
                                                         target_size=(img_size,img_size),
                                                         color_mode='grayscale',
                                                         batch_size=batch_size,
                                                         class_mode='categorical',
                                                         shuffle=False)

## Deep-CNN model building

In [ ]:
model = Sequential()

# Conv Block 1
model.add(Conv2D(64, (3,3), padding='same', input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Conv Block 2
model.add(Conv2D(128,(5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Conv Block 3
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Conv Block 3
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

# Fully connected Block 1
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

# Fully connected Block 2
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(7, activation='softmax'))

opt = Adam(lr=0.0005)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Fitting the model and saving the weights in .h5 file

In [ ]:
epochs = 25
steps_per_epoch= train_generator.n//train_generator.batch_size
validation_steps = validation_generator.n//validation_generator.batch_size

checkpoint = ModelCheckpoint("facial_expression_recognition/fer_model_weights.h5",monitor='val_accuracy',
                            save_weights_only=True, mode='max',verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss' , factor=0.1, patience=2, min_lr=0.001,model='auto')

callbacks = [checkpoint, reduce_lr]

history = model.fit(
        x= train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_steps,
        callbacks=callbacks
)

If you want to save the model in .json file:

In [ ]:
model_json = model.to_json()
with open("model.json","w+") as json_file:
    json_file.write(model_json)

Calling the cascade classifier from OpenCV

In [3]:
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')

classifier = load_model('facial_expression_recognition/fer_model_weights.h5')

In [ ]:
num_classes = 6
img_rows, img_cols = 48, 48
batch_size = 16

validation_data_dir = 'facial_expression_recognition/test'

validation_datagen = ImageDataGenerator(rescale= 1./255)

validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        color_mode = 'grayscale',
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)


class_labels = validation_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())
print(class_labels)

### Function to detect faces in an image.

In [4]:
def face_detector(img):
    # Convert image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return (0, 0, 0, 0), np.zeros((48, 48), np.uint8), img

    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
        roi_gray = gray[y:y + h, x:x + w]

    try:
        roi_gray = cv2.resize(roi_gray, (48, 48), interpolation=cv2.INTER_AREA)
    except:
        return (x, w, y, h), np.zeros((48, 48), np.uint8), img
    return (x, w, y, h), roi_gray, img


### OpenCV video processing and emotion detection.

In [13]:
# Provide path of the video to detect emotions in it.
cap = cv2.VideoCapture('facial_expression_recognition/fer_1.mp4')

emotions = []

img_array = []

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
out = cv2.VideoWriter('output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

# Divides video into the frames
while(cap.isOpened()):

    ret, frame = cap.read()
    if ret == True:

        rect, face, image = face_detector(frame)
        
        if np.sum([face]) != 0.0:
            roi = face.astype("float") / 255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)

            # make a prediction on the ROI, then lookup the class
            preds = classifier.predict(roi)[0]
            label = class_labels[preds.argmax()]
            emotions.append(label)
            label_position = (rect[0] + int((rect[1] / 2)), rect[2] + 25)
            out.write(cv2.putText(image, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3))

    else:
        break

out.release()